In [2]:
import mlrose_hiive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, KFold, validation_curve, learning_curve
import warnings
import time
warnings.filterwarnings('ignore')
from random import randint


In [3]:
df = pd.read_csv('/Users/qingzhou/Desktop/GT_OMSCS/Machine_Learning_CS7641/assignment_1/winequality-white.csv')
df.isnull().sum()
x_features = ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol']
y_output = ['quality']
class_name = ['bad', 'good']
data = df
df_x, df_y = data[x_features], data[y_output]
df_y['quality'].mask(df['quality'] <=5, 0, inplace= True)
df_y['quality'].mask(df['quality'] >=6, 1, inplace= True)
sc = StandardScaler()
X_train, X_test, Y_train, Y_test = train_test_split(df_x, df_y, test_size=0.2)
X_train, X_test = sc.fit_transform(X_train), sc.fit_transform(X_test)



In [21]:
def nn(x_train, y_train, x, y, algo, restart, schedule, size, key, itr):
    if size:
        nn_model = mlrose_hiive.NeuralNetwork(hidden_nodes = [25, 2], algorithm = algo, activation='identity', pop_size = size, mutation_prob=0.1,
                                          max_iters=itr, learning_rate=1e-3, early_stopping = True, random_state=10, max_attempts = 1000)
        print('ga')
    elif schedule:  
        nn_model = mlrose_hiive.NeuralNetwork(hidden_nodes = [25, 2], algorithm = algo, activation='identity', schedule= schedule,
                                          max_iters=itr, learning_rate=1e-3, early_stopping = True, random_state=10, max_attempts = 1000)
        print('sa')
    elif restart:
        nn_model = mlrose_hiive.NeuralNetwork(hidden_nodes = [25, 2], algorithm = algo, activation='identity', restarts= restart,
                                          max_iters=itr, learning_rate=1e-3, early_stopping = True, random_state=10, max_attempts = 1000)
        print('rhc')
    
    start = time.time()
    model=nn_model.fit(x_train, y_train)
    print(model.loss)
    model_time[key].append(time.time()-start)
    
    pred_train = model.predict(x_train)
    y_acc_train = accuracy_score(y_train, pred_train)
    
    
    pred = model.predict(x)
    y_acc = accuracy_score(y, pred)
    model_acc[key]['train'].append(y_acc_train)
    model_acc[key]['test'].append(y_acc)
    
    

In [18]:
def plotting(labels, data, rang, x, y, title, file):
    plt.figure()
    for label, arr in zip(labels, data):
        plt.plot(rang, arr, label=label)
    plt.legend()
    plt.xlabel(x)
    plt.ylabel(y)
    plt.title(title)
    plt.savefig(file, dpi = 1200)

In [19]:
model_time = {0:[], 1:[], 2:[]}
model_acc = {0:{'train':[], 'test':[]}, 1:{'train':[], 'test':[]}, 
             2:{'train':[], 'test':[]}}

In [24]:
nn(X_train, Y_train, X_test, Y_test, 'random_hill_climb', 10, None, None, 0, 90100)
nn(X_train, Y_train, X_test, Y_test, 'simulated_annealing', None, mlrose_hiive.ExpDecay(), None, 1, 80100)
nn(X_train, Y_train, X_test, Y_test, 'genetic_alg', None, None, 200, 2, 10100)

rhc
0.5577380355904562
sa
1.2517225562198702
ga
8.136747720096363
